In [1]:
# Benötigte Bibliotheken
import pandas as pd
import requests 
import io
import os
from os.path import join
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen
import datetime
from random import sample

# Vorbereitung der Datensätze für die live hack session

## Datenbeschaffung aus öffentlichen Quellen

In [2]:
# Ordner, in dem wir die Daten speichern
data_dir = 'daten_live_hack_session'
if not os.path.isdir(data_dir):
    os.mkdir(data_dir)

In [3]:
# URL zum Datensatz der normalen Twitter-User. Daten liegen als ZIP vor
resp = urlopen('http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip')
# Daten herunterladen
zipfile = ZipFile(BytesIO(resp.read()))
# ZIP entpacken und in Ordner "data_dir" speichern
zipfile.extractall(data_dir)

In [4]:
# unter diesem Pfad ist der Datensatz mit den "normalen" Tweets gespeichert
filename_normal = 'training.1600000.processed.noemoticon.csv'
pfad_normal = join(data_dir, filename_normal)

tweets_normal_raw = pd.read_csv(pfad_normal, \
                            names=['sentiment','ID','Date','Query','User','Tweet'],\
                            encoding="ISO-8859-1")
tweets_normal_raw.head()

,sentiment,ID,Date,Query,User,Tweet
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [5]:
# die URL unter der der Trump-Tweet-Datensatz zu finden ist
url_trump = "https://github.com/mkearney/trumptweets/raw/master/data/trumptweets-1515775693.tweets.csv"

# lies die Information von der in der URL hinterlegten Website aus
# und speichere sie in einem DataFrame. 
antwort_trump = requests.get(url_trump).content
inhalt_trump = io.StringIO(antwort_trump.decode('ISO-8859-1'))
tweets_trump_raw = pd.read_csv(inhalt_trump)
tweets_trump_raw.head()

/home/jana/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (15,30,31,32,36,37,38,42,43,44,45,46,47,50,51,52,56,57,58,59,60,61,62,63,64,65,66,67) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,status_id,created_at,user_id,screen_name,text,source,display_text_width,reply_to_status_id,reply_to_user_id,reply_to_screen_name,...,retweet_verified,place_url,place_name,place_full_name,place_type,country,country_code,geo_coords,coords_coords,bbox_coords
0,x1864367186,2009-05-20 22:29:47,x25073877,realDonaldTrump,Read a great interview with Donald Trump that ...,Twitter Web Client,112,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,x9273573134835712,2010-11-29 15:52:46,x25073877,realDonaldTrump,Congratulations to Evan Lysacek for being nomi...,Twitter Web Client,127,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,x29014512646,2010-10-28 18:53:40,x25073877,realDonaldTrump,I was on The View this morning. We talked abou...,Twitter Web Client,139,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,x7483813542232064,2010-11-24 17:20:54,x25073877,realDonaldTrump,Tomorrow night's episode of The Apprentice del...,Twitter Web Client,140,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,x5775731054,2009-11-16 21:06:10,x25073877,realDonaldTrump,Donald Trump Partners with TV1 on New Reality ...,Twitter Web Client,116,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# url der Troll tweets, mit einem Platzhalter "{}" für die Zahl.
# das Archiv beinhaltet 13 einzelne Teile des Datensatzes
url_troll = 'https://github.com/fivethirtyeight/russian-troll-tweets/raw/master/IRAhandle_tweets_{}.csv'
# Liste, in der wir alle heruntergeladenen Troll-Tweets speichern
all_troll_tweets_raw = []

# iteriere über die Zahlen 1 bis 13, um alle 13 Teile des Datensatzes herunterzuladen
for i in range(1, 14):
    url_troll = url_troll.format(i)
    antwort_troll = requests.get(url_troll).content
    inhalt_troll = io.StringIO(antwort_troll.decode('ISO-8859-1'))
    tweets = pd.read_csv(inhalt_troll)
    # füge den Datensatz-Teil der Liste hinzu
    all_troll_tweets_raw.append(tweets)
    print('Der {}. Datensatzteil beinhaltet {} Tweets.'.format(i, len(tweets)))
    
# erstelle aus den Teilen ein einziges DataFrame
all_troll_tweets_raw = pd.concat(all_troll_tweets_raw)

Der 1. Datensatzteil beinhaltet 243891 Tweets.
Der 2. Datensatzteil beinhaltet 243891 Tweets.
Der 3. Datensatzteil beinhaltet 243891 Tweets.
Der 4. Datensatzteil beinhaltet 243891 Tweets.
Der 5. Datensatzteil beinhaltet 243891 Tweets.
Der 6. Datensatzteil beinhaltet 243891 Tweets.
Der 7. Datensatzteil beinhaltet 243891 Tweets.
Der 8. Datensatzteil beinhaltet 243891 Tweets.
Der 9. Datensatzteil beinhaltet 243891 Tweets.
Der 10. Datensatzteil beinhaltet 243891 Tweets.
Der 11. Datensatzteil beinhaltet 243891 Tweets.
Der 12. Datensatzteil beinhaltet 243891 Tweets.
Der 13. Datensatzteil beinhaltet 243891 Tweets.


## Nicht benötigte Informationen löschen, Spaltennamen streamlinen

In [7]:
tweets_normal_raw.drop(columns = ['sentiment', 'ID', 'Query'], inplace = True)
tweets_normal_raw.head(2)

,Date,User,Tweet
0,Mon Apr 06 22:19:45 PDT 2009,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,Mon Apr 06 22:19:49 PDT 2009,scotthamilton,is upset that he can't update his Facebook by ...


In [8]:
tweets_trump_raw = tweets_trump_raw[['created_at', 'screen_name', 'text']]
tweets_trump_raw.rename(columns={'created_at':'Date', 'screen_name':'User', 'text':'Tweet'}, inplace=True)
tweets_trump_raw.head(2)

,Date,User,Tweet
0,2009-05-20 22:29:47,realDonaldTrump,Read a great interview with Donald Trump that ...
1,2010-11-29 15:52:46,realDonaldTrump,Congratulations to Evan Lysacek for being nomi...


In [10]:
# wir wollen nur englischsprachige Tweets, damit die Sprache vergleichbar mit
# den anderen beiden Datensätzen ist
all_troll_tweets_raw = all_troll_tweets_raw[all_troll_tweets_raw['language'] == 'English']

In [11]:
all_troll_tweets_raw = all_troll_tweets_raw[['publish_date','author', 'content']]
all_troll_tweets_raw.rename(columns = {'publish_date': 'Date', 'author':'User', 'content':'Tweet'}, inplace=True)
all_troll_tweets_raw.head(2)

,Date,User,Tweet
0,10/1/2017 19:58,10_GOP,"""We have a sitting Democrat US Senator on tria..."
1,10/1/2017 22:43,10_GOP,Marshawn Lynch arrives to game in anti-Trump s...


### Tweet-Längen berechnen und filtern

In [13]:
def calculate_tweet_length(df):
    
    # Liste, in der die Tweet-Längen gespeichert werden
    length = []
    
    # Iteriere über alle Tweets
    for tweet in df['Tweet']:
        # füge Tweet-Länge der Liste hinzu
        length.append(len(tweet))

    # erstelle eine neue Spalte im DataFrame "df",
    # indem der (zuvor noch nicht existenten) Spalte
    # mit dem namen "tweet_length" die Liste "length"
    # zugewiesen wird
    df['tweet_length'] = length
    
# wende die oben definierte Funktion auf alle DataFrames an
calculate_tweet_length(tweets_normal_raw)
calculate_tweet_length(tweets_trump_raw)
calculate_tweet_length(all_troll_tweets_raw)

In [14]:
# entferne alle Tweets mit > 140 Zeichen
tweets_normal_raw = tweets_normal_raw[tweets_normal_raw['tweet_length'] <= 140]
tweets_trump_raw = tweets_trump_raw[tweets_trump_raw['tweet_length'] <= 140]
all_troll_tweets_raw = all_troll_tweets_raw[all_troll_tweets_raw['tweet_length'] <= 140]

### Wortanzahl berechnen

In [15]:
def calculate_word_number(df):
    
    # Liste, in der die Wort-Längen gespeichert werden
    word_number = []
    
    # Iteriere über alle Tweets
    for tweet in df['Tweet']:
        # wir gehen davon aus, dass einzelne Worte durch
        # Leerzeichen getrennt werden
        words = tweet.split(' ')
        word_number.append(len(words))

    # erstelle eine neue Spalte im DataFrame "df",
    # indem der (zuvor noch nicht existenten) Spalte
    # mit dem namen "word_number" die Liste word_number
    # zugewiesen wird
    df['word_number'] = word_number
    
calculate_word_number(tweets_normal_raw)
calculate_word_number(tweets_trump_raw)
calculate_word_number(all_troll_tweets_raw)

In [16]:
tweets_normal_raw.head()

,Date,User,Tweet,tweet_length,word_number
0,Mon Apr 06 22:19:45 PDT 2009,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",115,20
1,Mon Apr 06 22:19:49 PDT 2009,scotthamilton,is upset that he can't update his Facebook by ...,111,22
2,Mon Apr 06 22:19:53 PDT 2009,mattycus,@Kenichan I dived many times for the ball. Man...,89,19
3,Mon Apr 06 22:19:57 PDT 2009,ElleCTF,my whole body feels itchy and like its on fire,47,11
4,Mon Apr 06 22:19:57 PDT 2009,Karoli,"@nationwideclass no, it's not behaving at all....",111,22


### Datumsangaben aufräumen

In [17]:
# definiere die Funktion "clean_date", die als Input
# einen String bekommt, der eine Datums- und Zeitangabe
# enthält, sowie den typ (trump, normal, troll) des Inputs,
# um den String entsprechend zu formatieren
def clean_date(date_time_raw, typ):
    
    # dictionary in dem jeder Monatsabkürzung eine Ganzzahl
    # zugewiesen wird (im normalen Tweet Datensatz liegen die
    # Monatsangaben als String vor)
    month_dict = {'Jan':1, 'Feb':2, 'Mar':3, 'Apr':4,\
                 'May':5, 'Jun':6, 'Jul':7, 'Aug':8,\
                 'Sep':9, 'Oct':10, 'Nov':11, 'Dec':12}
    
    # überprüfe den Typ der Eingabe
    # Timestamps aus dem Troll-Datensatz folgen dem
    # Muster "monat/tag/jahr stunde:minute"
    if typ == 'troll':
        # teile den String entlang des Leerzeichens in Datum und Zeit
        date, time = date_time_raw.split(' ')
        # teile das Datum entlang des forward-slash in Monat, Tag und Jahr
        month, day, year = date.split('/')
        # teile die Zeit entlang des Doppelpunktes in Stunde und Minute
        hour, minute = time.split(':')
    
    # Timestamps aus dem normalen Datensatz folgen dem
    # Muster "weekday month day hour:minute:second timezone year"
    elif typ == 'normal':
        # teile den String entlang des Leerzeichens in Wochentag,
        # Monat (String-Abkürzung), Tag, Zeit, Zeitzone und Jahr
        weekday, month, day, time, zone, year = date_time_raw.split(' ')
        # teile die Zeit entlang des Doppelpunktes in Stunde, Minute und Sekunde
        hour, minute, second = time.split(':')
        # konvertiere die Monatsangabe mit dem "month_dict" in eine Zahl
        month = month_dict[month]
    
    # Timestamps aus dem Trump-Datensatz folgen dem
    # Muster "year-month-day hour:minute:second"
    elif typ == 'trump':
        # teile den String entlang des Leerzeichens in Datum und Zeit
        date, time = date_time_raw.split(' ')
        # teile das Datum entlang des Bindestrichs in Jahr, Monat und Tag
        year, month, day = date.split('-')
        # teile die Zeit entlang des Doppelpunktes in Stunde, Minute und Sekunde
        hour, minute, second = time.split(':')
    
    # falls die Funktion mit einem unbekannten Typ benutzt
    # wird, geben wir eine Fehlermeldung aus und "None" zurück
    else:
        print('unknown Tweet type!')
        return None
    
    # wir verwandeln Jahr, Monat, Tag, Stunde und Minute in Ganzzahlen
    # da der Datensatz mit den Troll-Tweets keine Informationen über die
    # Sekunden enthält, lassen wir die Sekunden weg und geben uns mit
    # einer minuten-genauen Auflösung der Zeit zufrieden
    year = int(year)
    month = int(month)
    day = int(day)
    hour = int(hour)
    minute = int(minute)
    
    # erstelle ein datetime-Objekt aus Datum und Zeit
    date_time = datetime.datetime(year, month, day, hour, minute)
    
    # gib das datetime-Objekt zurück
    return date_time
        

In [18]:
# teste die Funktion mit Einträgen aus allen drei Datensätzen
trump_test = tweets_trump_raw['Date'][0:1].item()
normal_test = tweets_normal_raw['Date'][0:1].item()
troll_test = all_troll_tweets_raw['Date'][0:1].item()

clean_trump = clean_date(trump_test, 'trump')
print(clean_trump)
clean_normal = clean_date(normal_test, 'normal')
print(clean_normal)
clean_troll = clean_date(troll_test, 'troll')
print(clean_troll)

2009-05-20 22:29:00
2009-04-06 22:19:00
2017-10-01 22:43:00


In [19]:
# konvertiere alle Datumsangaben in den drei Datensätzen in datetime-Objekte
tweets_trump_raw['Date'] = tweets_trump_raw['Date'].apply(clean_date, args=['trump'])
tweets_normal_raw['Date'] = tweets_normal_raw['Date'].apply(clean_date, args=['normal'])
all_troll_tweets_raw['Date'] = all_troll_tweets_raw['Date'].apply(clean_date, args=['troll'])

In [20]:
tweets_trump_raw.sort_values(by='Date')
tweets_trump_raw.head()

,Date,User,Tweet,tweet_length,word_number
0,2009-05-20 22:29:00,realDonaldTrump,Read a great interview with Donald Trump that ...,112,16
1,2010-11-29 15:52:00,realDonaldTrump,Congratulations to Evan Lysacek for being nomi...,127,23
2,2010-10-28 18:53:00,realDonaldTrump,I was on The View this morning. We talked abou...,139,23
3,2010-11-24 17:20:00,realDonaldTrump,Tomorrow night's episode of The Apprentice del...,140,23
4,2009-11-16 21:06:00,realDonaldTrump,Donald Trump Partners with TV1 on New Reality ...,116,14


### Für die Trump-Tweets die Zeit seit dem ersten Tweet (in Stunden) berechnen

In [21]:
# das ist das Format des Strings mit der Zeitinformation, also
# Jahr-Monat-Tag Stunde:Minute:Sekunde
fmt = '%Y-%m-%d %H:%M:%S'

# wir verwandeln die Einträge der Spalte 'Date' in datetime-Objekte
#tweets_trump_raw['Date_new'] = [datetime.datetime.strptime(date, fmt) for date in tweets_trump_raw['Date']]

# berechnen der Zeitabstände
zeitabstand = [(date - tweets_trump_raw['Date'].min()).total_seconds() for date in tweets_trump_raw['Date']]

# Hinzufügen der Spalte
tweets_trump_raw['hours_since_first_tweet'] = zeitabstand

# bei genauerem Nachdenken stellt sich die Darstellung in Sekunden als
# nicht so praktikabel heraus, deswegen teilen wir noch einmal durch
# 60 (Sekunden) mal 60 (Minuten), um den Zeitabstand in Stunden anzugeben
tweets_trump_raw['hours_since_first_tweet'] = tweets_trump_raw['hours_since_first_tweet'] / (60*60)

# das können wir so einfach tun, da die in der Spalte "Date" gespeicherten
# datetime-Objekte ein Attribut .year haben, das uns direkten Zugriff auf das
# Jahr gibt (ähnlich übrigens mit Monat und Tag).
tweets_trump_raw['Year'] = [date.year for date in tweets_trump_raw['Date']]
tweets_trump_raw['Month'] = [date.month for date in tweets_trump_raw['Date']]
tweets_trump_raw['Hour'] = [date.hour for date in tweets_trump_raw['Date']]
tweets_trump_raw['Minute'] = [date.minute for date in tweets_trump_raw['Date']]

# Sortieren Tabelle anhand der Spalte "timedelta"
tweets_trump_raw.sort_values('hours_since_first_tweet',inplace=True)

tweets_trump_raw.head()

,Date,User,Tweet,tweet_length,word_number,hours_since_first_tweet,Year,Month,Hour,Minute
18,2009-05-04 18:54:00,realDonaldTrump,Be sure to tune in and watch Donald Trump on L...,117,23,0.000000,2009,5,18,54
48,2009-05-05 01:00:00,realDonaldTrump,Donald Trump will be appearing on The View tom...,131,22,6.100000,2009,5,1,0
56,2009-05-08 13:38:00,realDonaldTrump,Donald Trump reads Top Ten Financial Tips on L...,116,17,90.733333,2009,5,13,38
80,2009-05-08 20:40:00,realDonaldTrump,New Blog Post: Celebrity Apprentice Finale and...,103,13,97.766667,2009,5,20,40
78,2009-05-12 14:07:00,realDonaldTrump,"""My persona will never be that of a wallflower...",111,21,187.216667,2009,5,14,7


### Daten subsamplen um Anzahl auf Anzahl der Trump-Tweets zu reduzieren

In [23]:
tweets_trump_raw.reset_index(inplace=True, drop=True)
tweets_normal_raw.reset_index(inplace=True, drop=True)
all_troll_tweets_raw.reset_index(inplace=True, drop=True)

In [25]:
tweets_trump_raw.head(2)

,Date,User,Tweet,tweet_length,word_number,hours_since_first_tweet,Year,Month,Hour,Minute
0,2009-05-04 18:54:00,realDonaldTrump,Be sure to tune in and watch Donald Trump on L...,117,23,0.0,2009,5,18,54
1,2009-05-05 01:00:00,realDonaldTrump,Donald Trump will be appearing on The View tom...,131,22,6.1,2009,5,1,0


In [26]:
N = len(tweets_trump_raw)
# draw a random sample from the index list
random_sample_mask = sample(list(tweets_normal_raw.index), N)
tweets_normal = tweets_normal_raw[tweets_normal_raw.index.isin(random_sample_mask)]

In [27]:
random_sample_mask = sample(list(all_troll_tweets_raw.index), N)
tweets_troll = all_troll_tweets_raw[all_troll_tweets_raw.index.isin(random_sample_mask)]

In [28]:
len(tweets_normal)

30311

In [29]:
len(tweets_trump_raw)

30311

In [30]:
len(tweets_troll)

30311

### Spaltennamen umsortieren und DataFrames speichern

In [31]:
cols = ['User', 'Tweet', 'tweet_length', 'word_number', 'Date', 'Year', 'Month', 'Hour', 'hours_since_first_tweet']
tweets_trump = tweets_trump_raw[cols]
tweets_trump.head(2)

,User,Tweet,tweet_length,word_number,Date,Year,Month,Hour,hours_since_first_tweet
0,realDonaldTrump,Be sure to tune in and watch Donald Trump on L...,117,23,2009-05-04 18:54:00,2009,5,18,0.0
1,realDonaldTrump,Donald Trump will be appearing on The View tom...,131,22,2009-05-05 01:00:00,2009,5,1,6.1


In [32]:
cols = ['User', 'Tweet', 'tweet_length', 'word_number', 'Date']
tweets_troll = tweets_troll[cols]
tweets_normal = tweets_normal[cols]

In [38]:
tweets_trump.to_excel(join(data_dir, 'tweets_trump.xlsx'), index=False)
tweets_troll.to_excel(join(data_dir, 'tweets_troll_subset.xlsx'), index=False)
tweets_normal.to_excel(join(data_dir, 'tweets_normal_subset.xlsx'), index=False)